# **STT dan TTS**

Speech-to-Text (STT) menggunakan package "SpeechRecognition"

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1ktPjohdZX7Q3kSR1vaURGrEZQBq_1tl6?usp=sharing)

*Oleh : Sinubi*

## Persiapan

In [1]:
# Menginstall Package "ffmpeg-python"
!pip install ffmpeg-python==0.2.0 -q
print("Done !")

Done !


In [2]:
# Menginstall Package "SpeechRecognition"
!pip install SpeechRecognition==3.10.1 -q
print("Done !")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 5.9 MB/s eta 0:00:00
Done !


In [3]:
# Import Packages/Libraries yang diperlukan
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import io

import numpy as np

import scipy
from scipy.io.wavfile import read as wav_read

import ffmpeg

import speech_recognition as sr

In [4]:
# Menampilkan Versi Package "ffmpeg-python"
!pip show "ffmpeg-python"

Name: ffmpeg-python
Version: 0.2.0
Summary: Python bindings for FFmpeg - with complex filtering support
Home-page: https://github.com/kkroening/ffmpeg-python
Author: Karl Kroening
Author-email: karlk@kralnet.us
License: UNKNOWN
Location: /usr/local/lib/python3.10/dist-packages
Requires: future
Required-by: 


In [5]:
# Menampilkan Versi Packages lainnya
print("Versi SpeechRecognition : ", sr.__version__)
print("Versi Numpy             : ", np.__version__)
print("Versi Scipy             : ", scipy.__version__)

Versi SpeechRecognition :  3.10.1
Versi Numpy             :  1.23.5
Versi Scipy             :  1.11.4


In [6]:
# Fungsi JavaScript untuk Perekaman Audio

AUDIO_HTML = """
<script>

var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Tekan untuk Merekam Suara");

my_btn.appendChild(t);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    mimeType : 'audio/webm;codecs=opus'
  };

  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data);
    reader.onloadend = function() {
      base64data = reader.result;
    }
  };

  recorder.start();
};

recordButton.innerText = "Merekam Suara ... Tekan Tombol untuk Selesai Merekam !";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);

function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Menyimpan Rekaman Suara. Silahkan Tunggu !"
  }
}

function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
  recordButton.onclick = ()=>{
    toggleRecording()

    sleep(2000).then(() => {
      resolve(base64data.toString())
    });
  }
});

</script>
"""

In [7]:
# Fungsi Python untuk Perekaman Audio

def rekam_audio():
  display(HTML(AUDIO_HTML))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])

  process = (ffmpeg
    .input('pipe:0')
    .output('pipe:1', format='wav')
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
  )
  output, err = process.communicate(input=binary)

  riff_chunk_size = len(output) - 8
  # Break up the chunk size into four bytes, held in b.
  q = riff_chunk_size
  b = []
  for i in range(4):
      q, r = divmod(q, 256)
      b.append(r)

  # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
  riff = output[:4] + bytes(b) + output[8:]

  sr, audio = wav_read(io.BytesIO(riff))

  return audio, sr

## Speech-to-Text (STT)

### Merekam Audio

In [8]:
# Merekam Audio
audio, sr_ = rekam_audio()

In [9]:
# Menyimpan Hasil Rekaman Audio (Format File Audio .wav)
scipy.io.wavfile.write('rekaman.wav', sr_, audio)

In [10]:
# Memutar Kembali Hasil Rekaman Audio
sound_file = 'rekaman.wav'
Audio(sound_file, autoplay = True)

### Menghasilkan *Text* dari Rekaman Audio atau *Speech* (*Audio Transcription*)

In [11]:
# Audio Transcription
r = sr.Recognizer()

audio_file = sr.AudioFile('rekaman.wav')
with audio_file as source:
    audio = r.record(source)

print(r.recognize_google(audio, language="id-ID"))

tes tes 123


# Referensi Eksternal

*   https://blog.addpipe.com/recording-audio-in-the-browser-using-pure-html5-and-minimal-javascript/
*   https://stackoverflow.com/a/18650249
*   https://hacks.mozilla.org/2014/06/easy-audio-capture-with-the-mediarecorder-api/
*   https://air.ghost.io/recording-to-an-audio-file-using-html5-and-js/
*   https://stackoverflow.com/a/49019356
*   https://colab.research.google.com/drive/1Z6VIRZ_sX314hyev3Gm5gBqvm1wQVo-a

